# AHAB app Functions

In this notebook I will be testing the different functions needed to build up the app

In [47]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz, process
import json
from collections import Counter
import folium

In [2]:
with open('../data/CLEAN/audio_df.json', 'r') as audio:
    audio_products = json.load(audio)

with open('../data/CLEAN/camera_df.json', 'r') as cameras:
    camera_products = json.load(cameras)

with open('../data/CLEAN/lens_df.json', 'r') as lenses:
    lens_products = json.load(lenses)

with open('../data/CLEAN/lights_df.json', 'r') as lights:
    light_products = json.load(lights)

with open('../data/CLEAN/rental_places.json', 'r') as places:
    rental_places = json.load(places)

In [3]:
audiodf = pd.DataFrame(audio_products)
camerasdf = pd.DataFrame(camera_products)
lensesdf = pd.DataFrame(lens_products)
lightsdf = pd.DataFrame(light_products)
rentaldf = pd.DataFrame(rental_places)

In [4]:
rentaldf

,rental_place_id,rental_place_name,address,latitude,longitude,email,phone,website
0,1,WELAB,"C. de Albasanz, 59, 28037, Madrid",40.43856,-3.62394,info@welab.es,0034 911823355,https://www.welabplus.com
1,2,RC SERVICE,"Avda. Prado del Espino, 1, 28660, Boadilla del...",40.39606,-3.85031,info@rcservice.es,0034 911855868,https://www.rcservice.es
2,3,OVIDE,"Av. Montes de Oca, 19, Nave 23, 28703, San Seb...",40.55172,-3.61149,madrid@ovide.com,0034 915670407,https://www.ovide.com/


In [5]:
user_input_list = input('Feed me your needs: ').lower().split(',')

In [11]:
user_input_list

['arri alexa 35',
 ' arri/zeiss masterprime 18mm t1.3',
 ' zoom h6',
 ' nanlite pavotube 15c']

In [22]:
def find_products_in_dataframes(products):
    found_products_dict = {category: [] for category in ['Audio', 'Cameras', 'Lenses', 'Lights']}
    
    for category, df in zip(['Audio', 'Cameras', 'Lenses', 'Lights'], [audiodf, camerasdf, lensesdf, lightsdf]):
        found_products = []
        
        for product in products:
            # Filter by similarity and retrieve name and rental_place_id
            found = df[df['name'].apply(lambda x: fuzz.partial_ratio(x, product)) > 95][['name', 'rental_place_id']]
            found_products.extend(found.to_dict('records'))
        
        found_products_dict[category] = found_products
    
    return found_products_dict


In [23]:
user_input_list

['arri alexa 35',
 ' arri/zeiss masterprime 18mm t1.3',
 ' zoom h6',
 ' nanlite pavotube 15c']

In [24]:
find_products_in_dataframes(user_input_list)

{'Audio': [{'name': 'zoom h6', 'rental_place_id': 1},
  {'name': 'mezclador/grabador zoom h6', 'rental_place_id': 3}],
 'Cameras': [{'name': 'arri alexa 35', 'rental_place_id': 1},
  {'name': 'arri alexa 35', 'rental_place_id': 2},
  {'name': ' arri alexa 35 4k', 'rental_place_id': 3}],
 'Lenses': [{'name': ' arri/zeiss masterprime 18mm t 1.3',
   'rental_place_id': 2}],
 'Lights': [{'name': 'nanlite pavotube 15c', 'rental_place_id': 1}]}

In [29]:
product_dict = {'Audio': [{'name': 'zoom h6', 'rental_place_id': 1},
                          {'name': 'mezclador/grabador zoom h6', 'rental_place_id': 3}],
                'Cameras': [{'name': 'arri alexa 35', 'rental_place_id': 1},
                            {'name': 'arri alexa 35', 'rental_place_id': 2},
                            {'name': ' arri alexa 35 4k', 'rental_place_id': 3}],
                'Lenses': [{'name': ' arri/zeiss masterprime 18mm t 1.3', 'rental_place_id': 2}],
                'Lights': [{'name': 'nanlite pavotube 15c', 'rental_place_id': 1}]}

In [39]:
# Function to find the most common rental place ID

def find_most_common_place_id(found_products_dict):
    all_ids = [item['rental_place_id'] for sublist in found_products_dict.values() for item in sublist]
    count_ids = Counter(all_ids)
    
    most_common_id = None
    if count_ids:  # Check if count_ids is not empty
        most_common_id = count_ids.most_common(1)[0][0]
    
    return most_common_id

In [40]:
find_most_common_place_id(product_dict)

1

In [41]:
# Function to get rental place info based on the most common ID
def get_rental_place_info(most_common_id):
    
    return rentaldf[rentaldf['rental_place_id'] == most_common_id]

In [42]:
get_rental_place_info('1')

,rental_place_id,rental_place_name,address,latitude,longitude,email,phone,website
0,1,WELAB,"C. de Albasanz, 59, 28037, Madrid",40.43856,-3.62394,info@welab.es,0034 911823355,https://www.welabplus.com


In [71]:
class AHABFinder:
    def __init__(self, audiodf, camerasdf, lensesdf, lightsdf, rentaldf):
        
        self.audiodf = audiodf
        self.camerasdf = camerasdf
        self.lensesdf = lensesdf
        self.lightsdf = lightsdf
        self.rentaldf = rentaldf

    def find_products_in_dataframes(self, products):
        found_products_dict = {category: [] for category in ['Audio', 'Cameras', 'Lenses', 'Lights']}
        
        for category, df in zip(['Audio', 'Cameras', 'Lenses', 'Lights'], [self.audiodf, self.camerasdf, self.lensesdf, self.lightsdf]):
            found_products = []
            
            for product in products:
                # Filter by similarity and retrieve name and rental_place_id
                found = df[df['name'].apply(lambda x: fuzz.partial_ratio(x, product)) > 95][['name', 'rental_place_id']]
                found_products.extend(found.to_dict('records'))
            
            found_products_dict[category] = found_products
        
        return found_products_dict


    # Function to find the most common rental place ID

    def find_most_common_place_id(self, found_products_dict):
        all_ids = [item['rental_place_id'] for sublist in found_products_dict.values() for item in sublist]
        count_ids = Counter(all_ids)
        
        most_common_id = None
        if count_ids:  # Check if count_ids is not empty
            most_common_id = count_ids.most_common(1)[0][0]
        
        return most_common_id

    # Function to retrieve the rental_place info

    def get_rental_place_info(self, rental_place_id):
        
        return self.rentaldf[self.rentaldf['rental_place_id'] == str(rental_place_id)]#[['rental_place_name', 'address', 'email', 'phone', 'website']]


    # Function to display the rental place on a map.
    def display_map(self, df):
    
        map = folium.Map(location=[df['latitude'], df['longitude']], zoom_start=10.5)  # Initial map centered at a location

        # Add markers for rental shop locations
    
        lat = df['latitude']
        lon = df['longitude']
        name = df['rental_place_name']
        address = df['address']
        phone = df['phone']
        popup = folium.Popup(f"Company: {name} <br> Address: {address} <br> Phone: {phone}", min_width=300, max_width=300)
        folium.Marker([lat, lon], popup = popup).add_to(map)

        return map


    # Function to retrieve the info we want to display

    def find_rental_place_for_products(self):
        
        user_input = input("Enter a list of products separated by commas: ")
        input_products = [prod.strip() for prod in user_input.lower().split(',')]
        
        found_products_dict = self.find_products_in_dataframes(input_products)
        most_common_id = self.find_most_common_place_id(found_products_dict)
        rental_place_info = self.get_rental_place_info(most_common_id)
        rental_mp = self.display_map(rental_place_info)
        
        return  found_products_dict, most_common_id, rental_place_info, rental_mp

In [70]:
finder = AHABFinder(audiodf, camerasdf, lensesdf, lightsdf, rentaldf)

result = finder.find_rental_place_for_products()

result

/usr/local/Caskroom/miniconda/base/envs/finalProject/lib/python3.11/site-packages/folium/utilities.py:69: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(coord)
/usr/local/Caskroom/miniconda/base/envs/finalProject/lib/python3.11/site-packages/folium/utilities.py:77: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if math.isnan(float(coord)):
/usr/local/Caskroom/miniconda/base/envs/finalProject/lib/python3.11/site-packages/folium/utilities.py:79: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return [float(x) for x in coords]


,latitude,longitude
0,40.43856,-3.62394


In [ ]:
prod = 'Arri alexa 35, Arri/zeiss masterprime 18mm t1.3, Zoom h6, Nanlite pavotube 15c'

In [51]:

rentaldf.loc[[1]]

,rental_place_id,rental_place_name,address,latitude,longitude,email,phone,website
1,2,RC SERVICE,"Avda. Prado del Espino, 1, 28660, Boadilla del...",40.39606,-3.85031,info@rcservice.es,0034 911855868,https://www.rcservice.es


In [54]:
def display_map(self, df):
    
    map = folium.Map(location=[df['latitude'], df['longitude']], zoom_start=10.5)  # Initial map centered at a location

    # Add markers for rental shop locations
  
    lat = df['latitude']
    lon = df['longitude']
    name = df['rental_place_name']
    address = df['address']
    phone = df['phone']
    popup = folium.Popup(f"Company: {name} <br> Address: {address} <br> Phone: {phone}", min_width=300, max_width=300)
    folium.Marker([lat, lon], popup = popup).add_to(map)

    return map

In [55]:
display_map(rentaldf.loc[[1]])

/usr/local/Caskroom/miniconda/base/envs/finalProject/lib/python3.11/site-packages/folium/utilities.py:69: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(coord)
/usr/local/Caskroom/miniconda/base/envs/finalProject/lib/python3.11/site-packages/folium/utilities.py:77: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if math.isnan(float(coord)):
/usr/local/Caskroom/miniconda/base/envs/finalProject/lib/python3.11/site-packages/folium/utilities.py:79: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return [float(x) for x in coords]
